In [118]:
import arcpy
from arcpy import env

In [119]:
input_table = "C:\\Users\\gsbhatia\\Desktop\\kernel_density\\kde_gaussian_group_1\\D3Grp1.csv"

In [120]:
cursor = arcpy.SearchCursor(input_table)

In [121]:
env.workspace = 'C:\\Users\\gsbhatia\\Desktop\\kernel_density'

In [122]:
# arcpy.MakeTableView_management(in_table=input_table, out_view='viewtemp')
out_dbf = 'C:\\Users\\gsbhatia\\Desktop\\kernel_density\\D3Grp1.dbf'
# arcpy.TableToTable_conversion('viewtemp', out_gdb, 'tempTable')

In [123]:
arcpy.CopyRows_management(input_table,out_dbf)

<Result 'C:\\Users\\gsbhatia\\Desktop\\kernel_density\\D3Grp1.dbf'>

In [124]:
out_folder = 'C:\\Users\\gsbhatia\\Desktop\\kernel_density'
out_location = 'C:\\Users\\gsbhatia\\Desktop\\kernel_density\\out.gdb'
arcpy.CreateFileGDB_management(out_folder, "out.gdb")
arcpy.TableToGeodatabase_conversion('D3Grp1.dbf', out_location)

<Result 'C:\\Users\\gsbhatia\\Desktop\\kernel_density\\out.gdb'>

In [142]:
tables = arcpy.ListTables()

In [143]:
tables

[]

In [135]:
arcpy.env.workspace = 'C:\\Users\\gsbhatia\\Desktop\\kernel_density\\out.gdb'
# spatial_references = arcpy.ListSpatialReferences()
# print(spatial_references)
sr_path = r'C:\Users\gsbhatia\AppData\Local\ESRI\ArcGISPro\Favorites\gb_coordinate.prj'
sr =  arcpy.SpatialReference(sr_path)
arcpy.management.XYTableToPoint(input_table, "D3Grp1","CentroidX", "CentroidY",coordinate_system=sr)

<Result 'C:\\Users\\gsbhatia\\Desktop\\kernel_density\\out.gdb\\D3Grp1'>

In [144]:
arcpy.env.workspace = 'C:\\Users\\gsbhatia\\Desktop\\kernel_density\\out.gdb'
featureclasses = arcpy.ListFeatureClasses()
print(featureclasses)

['D3Grp1', 'D3Grp1Results']


In [140]:
import os

try:
    # Set the workspace and input features.
    arcpy.env.workspace = out_location
    inputFeatures = 'D3Grp1'

    # Set the output workspace and output name.
    outws = out_location
    outputName = 'D3Grp1Results'

    # Set input features, dependent variable, and explanatory variable.
    depVar = 'tumor_kde'
    explVar = 'immune_kde'

    # Set number of neighbors and permutations.
    numNeighbors = 100
    numPerms = '99'

    # Choose to create popups.
    popUps = 'CREATE_POPUP'

    # Choose confidence level and apply False Discovery Rate correction.
    confLevel = '95%'
    fdr = 'APPLY_FDR'

    # Set the scaling factor.
    scaleFactor = 0.5
    spatial_ref = None
    # Run Local Bivariate Regression.
    arcpy.LocalBivariateRelationships_stats(inputFeatures, depVar, explVar, 
                                            os.path.join(outws, outputName), 
                                            numNeighbors, numPerms, popUps, 
                                            confLevel, fdr, scaleFactor)

except arcpy.ExecuteError:
    # If an error occurred when running the tool, print out the error message.
    print(arcpy.GetMessages())

In [156]:
out_table = 'sumstats'

intable = 'D3Grp1Results'
# for field in arcpy.ListFields(intable):
#     print(field.name)
    
stats = [['LBR_TYPE','COUNT'],['LBR_SIG','COUNT']]    
arcpy.Statistics_analysis(intable, out_table, stats)


<Result 'C:\\Users\\gsbhatia\\Desktop\\kernel_density\\out.gdb\\sumstats'>

In [159]:
import pandas 

# Get a list of field names to display
field_names = [i.name for i in arcpy.ListFields(intable) if i.type != 'OID']

# Open a cursor to extract results from stats table
cursor = arcpy.da.SearchCursor(intable, field_names)

# Create a pandas dataframe to display results
df = pandas.DataFrame(data=[row for row in cursor],
                      columns=field_names)

print(df)

                                        Shape  ...                                         HTML_CHART
0     (1127.6000003814697, 961.2999992370605)  ...  <html>\n  <head>\n    <meta charset = "utf-8">...
1     (1595.8999996185303, 969.7000007629395)  ...  <html>\n  <head>\n    <meta charset = "utf-8">...
2                 (1253.6999998092651, 968.5)  ...  <html>\n  <head>\n    <meta charset = "utf-8">...
3                 (1515.0, 971.8999996185303)  ...  <html>\n  <head>\n    <meta charset = "utf-8">...
4     (1210.3000001907349, 971.2000007629395)  ...  <html>\n  <head>\n    <meta charset = "utf-8">...
..                                        ...  ...                                                ...
995  (19489.699999809265, 3406.7999992370605)  ...  <html>\n  <head>\n    <meta charset = "utf-8">...
996               (19168.89999961853, 3407.0)  ...  <html>\n  <head>\n    <meta charset = "utf-8">...
997              (19566.800000190735, 3407.5)  ...  <html>\n  <head>\n    <meta ch

In [163]:
df['LBR_TYPE'].value_counts() / df.shape[0]

Positive Linear      0.372
Not Significant      0.306
Concave              0.142
Convex               0.102
Undefined Complex    0.078
Name: LBR_TYPE, dtype: float64